In [45]:
# !pip install openwakeword
# !pip install speechbrain
# !pip install datasets
# !pip install scipy matplotlib

In [46]:
import os
import collections
import numpy as np
from numpy.lib.format import open_memmap
from pathlib import Path

from onnxruntime.transformers.shape_infer_helper import file_path
from tqdm import tqdm
import openwakeword
# import openwakeword.data
import openwakeword.utils
import openwakeword.metrics


import scipy
import datasets
import matplotlib.pyplot as plt
import torch
from torch import nn
import IPython.display as ipd

import datasets
import os
import numpy as np
import scipy.io.wavfile
from tqdm import tqdm

import urllib.request


import warnings

warnings.filterwarnings("ignore")

In [47]:


ds = datasets.load_dataset(
    "speech_commands",
    "v0.02",
    split="test",
    streaming=True
)
ds_iter = iter(ds)

os.makedirs("./data/speech_commands_test", exist_ok=True)
limit = 4890

for i in tqdm(range(limit)):
    output_file = f"./data/speech_commands_test/{i:05d}.wav"

    if os.path.exists(output_file):
        continue

    example = next(ds_iter)
    wav_data = (example["audio"]["array"] * 32767).astype(np.int16) # Convert to 16-bit PCM Format
    scipy.io.wavfile.write(output_file, 16000, wav_data)

100%|██████████| 4890/4890 [00:00<00:00, 16652.73it/s]


# ⇲ Compute Audio Embeddings

Create audio pre-processing object to get openwakeword audio embeddings

In [48]:
model_dir = "./resources/models"
os.makedirs(model_dir, exist_ok=True)

List Model and URL corresponding

In [49]:
models = {
    "embedding_model.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx",
    "embedding_model.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite",
    "melspectrogram.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx",
    "melspectrogram.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite"
}

In [50]:
for filename, url in models.items():
    file_path = os.path.join(model_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"Found {filename}.")


Found embedding_model.onnx.
Found embedding_model.tflite.
Found melspectrogram.onnx.
Found melspectrogram.tflite.


In [51]:
F = openwakeword.utils.AudioFeatures(
    melspec_model_path=f"{model_dir}/melspectrogram.onnx",
    embedding_model_path=f"{model_dir}/embedding_model.onnx",
    inference_framework="onnx",
)

## ⌀ Negative Clips

Get negative example paths, filtering out clips that are too long or too short

 pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cpu

In [52]:
import openwakeword.data

In [53]:
negative_clips, negative_durations = openwakeword.data.filter_audio_paths(
    [
        "./data/fma_sample",
        "./data/fsd50k_sample",
        "./data/speech_commands_test",
    ],
    min_length_secs=1.0, # minimum clip length in seconds
    max_length_secs=60*30, # maximum clip length in seconds
    duration_method="header" # use the file header to calculate duration
)

print(f"{len(negative_clips)} negative clips after filtering, representing ~{sum(negative_durations)//3600} hours")

200it [00:00, 6478.74it/s]
100%|██████████| 200/200 [00:01<00:00, 111.20it/s]
1000it [00:00, 13335.11it/s]
100%|██████████| 1000/1000 [00:04<00:00, 227.16it/s]
4890it [00:00, 56887.77it/s]
100%|██████████| 4890/4890 [00:07<00:00, 617.77it/s]

5987 negative clips after filtering, representing ~5.0 hours


Use HuggingFace datasets to load files from disk by batches, because:
- Load lazy
- Memmory-Mapping for doesn't load entire in RAM
- map() -> feature extraction, augmentation
- filter() -> remove audio too long/short

In [54]:
audio_dataset = datasets.Dataset.from_dict({
    "audio": negative_clips
})

In [55]:
audio_dataset = audio_dataset.cast_column(
    "audio",
    datasets.Audio(sampling_rate=16000)
)

- Get Audio Embeddings (features) for negative clips and save to .npy file
- Process files by batch and save to Numpy memory mapped file so that
- An array larger than the available system memory can be created

In [56]:
batch_size = 64 # number of files to load, compute features, and write to mmap at a time

In [57]:
clip_size = 3 # The desired window size (in seconds) for the trained openWakeWord model

In [58]:
N_total = int(sum(negative_durations)//clip_size) # Maximum number of rows in mmap file

N_total

6441

In [59]:
n_feature_cols = F.get_embedding_shape(clip_size)

n_feature_cols

(28, 96)

In [60]:
output_file = "negative_features.npy"

In [61]:
output_array_shape = (N_total, n_feature_cols[0], n_feature_cols[1])

output_array_shape

(6441, 28, 96)

Remove .npy before create mmap

In [62]:
for f in [output_file, output_file + ".npy"]:
    if os.path.exists(f):
        os.remove(f)

In [63]:
fp = open_memmap(
    output_file,
    mode="w+",
    dtype=np.float32,
    shape=output_array_shape
)

fp

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [64]:
row_counter = 0

In [65]:
print("\n===== START PROCESS =====")
print(f"Total rows: {audio_dataset.num_rows}")
print(f"Batch size: {batch_size}")
print(f"Expected batches: {int(np.ceil(audio_dataset.num_rows / batch_size))}")
print("")

for i in tqdm(np.arange(0, audio_dataset.num_rows, batch_size)):
    print(f"\n--- Batch index: {int(i / batch_size)} ---")

    print("Loading audio batch...")
    block = audio_dataset[i:i + batch_size]["audio"]
    print(f"Loaded {len(block)} clips")

    print("Converting to int16 arrays...")
    wav_data = [(j["array"] * 32767).astype(np.int16) for j in block]
    print("Conversion done.")

    print("Stacking clips (pad/trim)...")
    wav_data = openwakeword.data.stack_clips(
        wav_data,
        clip_size=16000 * clip_size
    ).astype(np.int16)
    print("Stack clips done.")

    print("Computing embeddings...")
    features = F.embed_clips(
        x=wav_data,
        batch_size=1024,
        ncpu=8
    )
    print(f"Embedding done. features.shape = {features.shape}")

    print("Saving to mmap...")

    rows_needed = features.shape[0]

    if row_counter + rows_needed > N_total:
        print(f"Final batch: writing {N_total - row_counter} rows then break.")
        fp[row_counter:N_total, :, :] = features[0:N_total - row_counter]
        fp.flush()
        break

    else:
        print(f"Writing rows {row_counter} → {row_counter + rows_needed}")
        fp[row_counter:row_counter + rows_needed, :, :] = features
        row_counter += rows_needed
        fp.flush()


===== START PROCESS =====
Total rows: 5987
Batch size: 64
Expected batches: 94



  0%|          | 0/94 [00:00<?, ?it/s]


--- Batch index: 0 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


  1%|          | 1/94 [00:06<10:38,  6.87s/it]

Embedding done. features.shape = (640, 28, 96)
Saving to mmap...
Writing rows 0 → 640

--- Batch index: 1 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


...

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Writing rows 6380 → 6402

--- Batch index: 89 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


 96%|█████████▌| 90/94 [01:16<00:01,  3.26it/s]

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Writing rows 6402 → 6424

--- Batch index: 90 ---
Loading audio batch...
Loaded 64 clips
Converting to int16 arrays...
Conversion done.
Stacking clips (pad/trim)...
Stack clips done.
Computing embeddings...


 96%|█████████▌| 90/94 [01:16<00:03,  1.17it/s]

Embedding done. features.shape = (22, 28, 96)
Saving to mmap...
Final batch: writing 17 rows then break.


In [66]:
print("\n===== Trimming mmap file =====")
openwakeword.data.trim_mmap(output_file)
print("===== FINISHED =====\n")


===== Trimming mmap file =====


Trimming empty rows: 7it [00:01,  4.76it/s]                       

===== FINISHED =====



Now we have all of the negative features prepared, and saved to fixed durations clips in Numpy array. For this data, the array is small at ~160MB, but in-practice the memory mapping allows to be very large.

## ✰ Negative Clips

The positive clips will be prepared in two way:
1. Mixing the synthetic positive clips with negative data at random SNRs to simulate noise data
2. Aligning the positive clips with background data such as that the end of the input window aligns with the end of the positive clip. This way the model will learn to predict the presence of the wakeword/phrase immediately after it is spoken.

In practice, there are other possible ways to augment the positive data (e.g, creating reverberation with room impulse response files, mixing with synthetic noise, etc.) but in pratice we have observed that mixing with realistic background data provides the best results.

After this preparation, the positive clips will be converted into the openWakeWord features in the same way as the negative files.

**Get positive example paths, filtering out clips that are too long or too short**

In [67]:
positive_clips, durations = openwakeword.data.filter_audio_paths(
    [
        "./data/turn_on_the_office_lights"
    ],
    min_length_secs=1.0,
    max_length_secs=2.0,
    duration_method="header" # use the file header to calculate duration
)

print(f"{len(positive_clips)} positive clips after filtering, representing ~{sum(durations)//3600} hours")

3388it [00:00, 11175.21it/s]
100%|██████████| 3388/3388 [00:26<00:00, 125.96it/s]

3203 positive clips after filtering, representing ~1.0 hours


Define starting point for each positive clip based on its length, so that each one ends between 0-200ms from the end of the total window size chosen for the model.

This results in the model being most confident in the prediction right after the end of the wakeword in the audio stream, reducing latency in operation.

Get start and end position for the positive audio in the full window

In [68]:
sr = 16000
total_length_seconds = 3 # must be some window length as that used for the negative examples
total_length = int(sr * total_length_seconds)

random delay (0-0.2s) -> make sure the audio don't need right position -> reduce overfitting

In [69]:
jitters = (np.random.uniform(0, 0.2, len(positive_clips)) * sr).astype(np.int32)

jitters

array([2095, 1494, 1934, ...,  500, 2201,  185],
      shape=(3203,), dtype=int32)

start position of wakeword for 3 seconds = total_length - (clip_length + jitter)


In [70]:
starts = [
    total_length
    -
    (
        int(np.ceil(i * sr)) + j
    ) for i,j in zip(durations, jitters)
]


starts

[np.int32(17299),
 np.int32(25143),
 np.int32(28233),
 np.int32(24940),
 np.int32(23750),
 np.int32(25444),
 np.int32(24960),
 np.int32(25562),
 np.int32(21872),
 np.int32(23157),
 np.int32(21154),
 np.int32(20644),
 np.int32(18381),
 np.int32(17686),
 np.int32(21227),
 np.int32(22864),
 np.int32(24021),
 np.int32(23330),
 np.int32(20297),
 np.int32(20653),
 np.int32(20692),
 np.int32(22218),
 np.int32(26625),
 np.int32(30312),
 np.int32(13911),
 np.int32(18013),
 np.int32(25166),
 np.int32(21778),
 np.int32(20872),
 np.int32(15585),
 np.int32(21245),
 np.int32(28610),
 np.int32(18309),
 np.int32(24545),
 np.int32(23233),
 np.int32(26866),
 np.int32(20321),
 np.int32(21804),
 np.int32(27507),
 np.int32(19126),
 np.int32(19475),
 np.int32(23735),
 np.int32(25364),
 np.int32(22741),
 np.int32(16092),
 np.int32(24663),
 np.int32(19958),
 np.int32(19805),
 np.int32(18974),
 np.int32(21658),
 np.int32(28217),
 np.int32(21222),
 np.int32(18151),
 np.int32(21389),
 np.int32(18552),
 np.int32(

end = clip_length + start_index

In [71]:
ends = [
    (int(i * sr) + j) for i, j in zip(durations, starts)
]

ends

[np.int32(45905),
 np.int32(46506),
 np.int32(46066),
 np.int32(47974),
 np.int32(47898),
 np.int32(47920),
 np.int32(46880),
 np.int32(45067),
 np.int32(45463),
 np.int32(45634),
 np.int32(46418),
 np.int32(46464),
 np.int32(45687),
 np.int32(46292),
 np.int32(46861),
 np.int32(47012),
 np.int32(47612),
 np.int32(44879),
 np.int32(46489),
 np.int32(44801),
 np.int32(47812),
 np.int32(47295),
 np.int32(46130),
 np.int32(47960),
 np.int32(45490),
 np.int32(45133),
 np.int32(47829),
 np.int32(46298),
 np.int32(47621),
 np.int32(44935),
 np.int32(46879),
 np.int32(46815),
 np.int32(46358),
 np.int32(45351),
 np.int32(45152),
 np.int32(45257),
 np.int32(45212),
 np.int32(46324),
 np.int32(45526),
 np.int32(46989),
 np.int32(46038),
 np.int32(45097),
 np.int32(46355),
 np.int32(45403),
 np.int32(46185),
 np.int32(47326),
 np.int32(45778),
 np.int32(47668),
 np.int32(46280),
 np.int32(47850),
 np.int32(47165),
 np.int32(47042),
 np.int32(45457),
 np.int32(45166),
 np.int32(45488),
 np.int32(

Create generator to mix the positive audio with background audio

In [72]:
batch_size = 8

In [73]:
mixing_generator = openwakeword.data.mix_clips_batch(
    foreground_clips = positive_clips,
    background_clips = negative_clips,
    combined_size = total_length,
    batch_size = batch_size,
    snr_low = 5,
    snr_high = 15,
    start_index = starts,
    volume_augmentation = True # Randomly scale the volume of the audio after mixing
)

(Optionally) listen to mixed clips to confirm that the mixing appears correctly

In [74]:
mixed_clips, labels, background_clips = next(mixing_generator)
ipd.display(ipd.Audio(mixed_clips[0], rate=16000, normalize=True, autoplay=False))

Iterate through the mixing generator, computing audio features for positive examples and saving them

In [75]:
N_total = len(positive_clips) # Maximum number of rows in mmap files

N_total

3203

In [76]:
n_feature_cols = F.get_embedding_shape(
    total_length_seconds
)

n_feature_cols

(28, 96)

In [77]:
output_file = "turn_on_the_office_lights_features.npy"

In [78]:
output_array_shape = (N_total, n_feature_cols[0], n_feature_cols[1])

output_array_shape

(3203, 28, 96)

Remove .npy before create mmap


In [79]:
for f in [output_file, output_file + ".npy"]:
    if os.path.exists(f):
        os.remove(f)

In [80]:
fp = open_memmap(
    output_file,
    "w+",
    dtype=np.float32,
    shape=output_array_shape
)

fp

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [81]:
row_counter = 0

In [82]:
import numpy as np
from tqdm import tqdm

print("===== START FEATURE EXTRACTION =====")
print(f"Total rows to process: {N_total}")
print(f"Batch size: {batch_size}")
print(f"Expected batches: {N_total // batch_size}")

row_counter = 0

for batch_idx, batch in enumerate(tqdm(mixing_generator, total=N_total//batch_size, desc="Processing batches")):

    # Unpack batch
    audio_batch, lbls, background = batch[0], batch[1], batch[2]
    print(f"\n--- Batch index: {batch_idx} ---")
    print(f"Audio batch shape: {audio_batch.shape}")

    # Compute embeddings
    print("Computing embeddings...")
    features = F.embed_clips(
        audio_batch,
        batch_size=256
    )
    print(f"Embeddings shape: {features.shape}")

    # Save computed features to memory-mapped file
    start_row = row_counter
    end_row = row_counter + features.shape[0]

    if end_row > N_total:
        # Truncate if last batch exceeds total rows
        print(f"Saving final partial batch: rows {start_row} → {N_total}")
        fp[start_row:N_total, :, :] = features[:N_total - start_row]
        fp.flush()
        row_counter = N_total
        break
    else:
        print(f"Saving batch rows: {start_row} → {end_row}")
        fp[start_row:end_row, :, :] = features
        fp.flush()
        row_counter = end_row

print("\n===== FEATURE EXTRACTION FINISHED =====")
print(f"Total rows written: {row_counter}")


===== START FEATURE EXTRACTION =====
Total rows to process: 3203
Batch size: 8
Expected batches: 400


Processing batches:   0%|          | 0/400 [00:00<?, ?it/s]


--- Batch index: 0 ---
Audio batch shape: (8, 48000)
Computing embeddings...


Processing batches:   0%|          | 1/400 [00:00<03:14,  2.05it/s]

Embeddings shape: (8, 28, 96)
Saving batch rows: 0 → 8

--- Batch index: 1 ---
Audio batch shape: (8, 48000)
Computing embeddings...


...

Embeddings shape: (8, 28, 96)
Saving batch rows: 3176 → 3184

--- Batch index: 398 ---
Audio batch shape: (8, 48000)
Computing embeddings...


Processing batches: 100%|█████████▉| 399/400 [03:26<00:00,  2.00it/s]

Embeddings shape: (8, 28, 96)
Saving batch rows: 3184 → 3192

--- Batch index: 399 ---
Audio batch shape: (3, 48000)
Computing embeddings...


Processing batches: 100%|██████████| 400/400 [03:26<00:00,  1.93it/s]

Embeddings shape: (3, 28, 96)
Saving batch rows: 3192 → 3195

===== FEATURE EXTRACTION FINISHED =====
Total rows written: 3195


Trip empty rows from the mmapped array

In [84]:
print("\n===== Trimming mmap file =====")
openwakeword.data.trim_mmap(output_file)
print("===== FINISHED =====\n")


===== Trimming mmap file =====


Trimming empty rows: 4it [00:00, 10.12it/s]                       


===== FINISHED =====



Alright! At this point the positive and negative features have been pre-computed and saved to disk, and now a model can be trained that takes these features and predicts whether the wakeword/phrase in present.